# PokerBot Championship Pipeline (Google Colab)

This notebook sets up and runs the full PokerBot championship pipeline in Colab:
1. Clone the GitHub repository
2. Install dependencies
3. Verify GPU availability
4. Generate training data
5. Train the model (GPU)
6. Validate results
7. Analyze and visualize results

## ⚙️ Setup Instructions
**IMPORTANT:** 
- Set Runtime Type to **GPU** (Runtime → Change runtime type → Hardware accelerator: GPU)
- Recommended: T4 or better for optimal performance
- Estimated total runtime: ~3-4 hours for medium training (5K samples)

## 📊 What This Notebook Does
- Trains a DeepStack neural network for poker decision-making
- Uses championship-level hyperparameters
- Generates comprehensive analysis reports
- Expected correlation: >0.85 (target for production use)

## 🚀 Enhanced Training Pipeline with Official DeepStack Data

This notebook has been optimized to train a DeepStack-style poker AI using:
1. **CFR-Generated Training Data**: Synthetic data from Counterfactual Regret Minimization
2. **Official DeepStack Championship Data**: Real hand histories from world-class matches
3. **Blended Approach**: Championship insights + CFR generation for best results

### 📊 Data Sources

**Official DeepStack Championship Data** (`/data/official_deepstack_handhistory/`):
- **DeepStack vs IFP Pros**: 44,000+ hands from professional matches
- **DeepStack vs LBR**: 80,000+ hands from benchmark testing
- **Formats**: ACPC logs, AIVAT analysis, PokerStars-compatible

**Extracted Insights**:
- Street distribution (39% preflop, 8% flop, 13% turn, 39% river)
- Betting patterns (pot-relative sizing per street)
- Recommended CFR iterations (2000-2500 for championship quality)
- Position-aware strategies

### 🎯 Training Profiles

| Profile | Samples | CFR Iters | Time | Expected Correlation |
|---------|---------|-----------|------|---------------------|
| **testing** | 1,000 | 500 | ~10 min | 0.40-0.50 |
| **development** | 10,000 | 1,500 | ~2 hours | 0.65-0.75 |
| **production** | 100,000 | 2,500 | ~24 hours | 0.75-0.85 |
| **championship** | 500,000 | 2,500 | ~5 days | >0.85 |

### 🔧 Key Features

✅ **Smart Data Detection**: Automatically uses championship data when available
✅ **Profile-Based Configuration**: Easy switching between training modes
✅ **Championship Bet Sizing**: Per-street pot-relative abstractions
✅ **Progress Monitoring**: Real-time training metrics and validation
✅ **GPU Acceleration**: Automatic GPU detection and usage
✅ **Comprehensive Visualization**: Loss curves, correlation plots, per-street analysis

### 📈 Expected Results

With **development** profile + championship data:
- Training time: ~2 hours (data) + ~1.5 hours (training)
- Validation correlation: 0.70-0.75
- Model size: ~5-10 MB
- Good for: Development, testing, initial deployment

With **production** profile + championship data:
- Training time: ~24 hours (data) + ~2-3 hours (training)
- Validation correlation: 0.80-0.85
- Model size: ~10-20 MB
- Good for: Production deployment, competitive play

### 🎓 Learning Resources

- **DeepStack Paper**: [Science 2017](https://www.science.org/doi/10.1126/science.aam6960)
- **CFR Tutorial**: Understanding counterfactual regret minimization
- **Hand History Analysis**: `data/handhistory_analysis.json`
- **Training Logs**: `models/reports/` (generated after training)

---

**Ready to begin? Run cells in order from top to bottom!** ⬇️

In [ ]:
# 1. Clone the PokerBot GitHub repository (with safety checks)
import os
import sys

# Check if already cloned to prevent re-cloning
if os.path.exists('pokerbot'):
    print('✓ Repository already exists, skipping clone.')
    %cd pokerbot
    !git pull origin main  # Update to latest version
else:
    print('Cloning PokerBot repository...')
    !git clone https://github.com/elliotttmiller/pokerbot.git
    %cd pokerbot
    print('✓ Repository cloned successfully')

# Verify we're in the right directory
if not os.path.exists('requirements.txt'):
    print('❌ ERROR: Not in the correct directory. Please restart and re-run.')
    sys.exit(1)

print(f'\n📁 Current directory: {os.getcwd()}')

In [ ]:
# 2. Install required dependencies with Colab-aware handling
import subprocess
import sys

print('Installing dependencies...')
print('This may take a few minutes.\n')

# Uninstall numpy and pandas first to prevent conflicts (Colab-specific)
print('Attempting to uninstall numpy and pandas to prevent conflicts...')
try:
    subprocess.run(
        [sys.executable, '-m', 'pip', 'uninstall', '-y', 'numpy', 'pandas'],
        capture_output=True,
        text=True,
        timeout=60
    )
    print('Uninstallation attempted.')
except Exception as e:
    print(f'Note: Pre-uninstall encountered an issue: {e}')

# Install with proper version constraints
try:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'],
        capture_output=True,
        text=True,
        timeout=300  # 5 minute timeout
    )
    
    if result.returncode == 0:
        print('✓ All dependencies installed successfully')
    else:
        # Show stderr but filter out common warnings (case-insensitive)
        stderr = result.stderr
        stderr_lower = stderr.lower()
        if 'warning' in stderr_lower and 'restart' in stderr_lower:
            print('⚠️  Note: Some packages were updated and require a runtime restart.')
        elif stderr:
            print(f'⚠️ Warning: Some packages may have had issues:')
            # Show last 1000 chars to avoid overwhelming output
            print(stderr[-1000:])
except subprocess.TimeoutExpired:
    print('❌ Installation timed out. Please try again.')
except Exception as e:
    print(f'❌ Error during installation: {e}')

print('\n' + '='*70)
print('⚠️  IMPORTANT: Runtime Restart Required')
print('='*70)
print('To ensure packages load correctly:')
print('  1. Runtime → Restart runtime')
print('  2. After restart, skip cells 1-2 and run from Cell 3')
print('  3. This ensures new numpy/pandas versions are properly loaded')
print('='*70)

# Verify critical packages (will likely fail until restart)
print('\n📦 Verifying critical packages...')
critical_packages = ['torch', 'numpy', 'pandas', 'matplotlib']
failed_packages = []
for package in critical_packages:
    try:
        __import__(package)
        print(f'  ✓ {package}')
    except (ImportError, ValueError) as import_error:
        # Provide more context about the error
        error_msg = str(import_error)[:100] if import_error else 'Unknown error'
        print(f'  ⚠️  {package} - Will work after runtime restart ({error_msg})')
        failed_packages.append(package)

if failed_packages:
    print(f'\n⚠️  {len(failed_packages)} package(s) need runtime restart to load properly.')
    print('This is normal! Just restart the runtime as instructed above.')
else:
    print('\n✓ All packages verified successfully!')


In [ ]:
# 3. Verify GPU availability and system resources
import torch

try:
    import psutil
except ImportError:
    !pip install -q psutil
    import psutil

print('=== SYSTEM RESOURCES ===' )
print(f'\n🖥️  CPU Cores: {psutil.cpu_count()}')
print(f'💾 RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB')
print(f'💿 Disk: {psutil.disk_usage("/").free / (1024**3):.1f} GB free')

print('\n=== GPU STATUS ===')
if torch.cuda.is_available():
    print(f'✓ GPU Available: {torch.cuda.get_device_name(0)}')
    print(f'✓ CUDA Version: {torch.version.cuda}')
    print(f'✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB')
    USE_GPU = True
    print('\n✅ Ready for GPU-accelerated training!')
else:
    print('❌ No GPU detected!')
    print('\n⚠️  WARNING: Training will be MUCH slower on CPU.')
    print('   Recommendation: Go to Runtime → Change runtime type → GPU')
    USE_GPU = False
    
    response = input('\nContinue without GPU? (yes/no): ')
    if response.lower() != 'yes':
        print('Please enable GPU and restart the notebook.')
        import sys
        sys.exit(0)

print('\n=== CONFIGURATION ===')
print(f'GPU Training: {"Enabled" if USE_GPU else "Disabled"}')
print('Ready to proceed! ✓')

In [ ]:
# 3.5 (Optional) Analyze Official DeepStack Hand History Data
import os
import json

print('=== OFFICIAL DEEPSTACK DATA ANALYTICS ===')
print('\nThis optional step analyzes official DeepStack championship hand histories')
print('to extract betting patterns, ranges, and strategies for training optimization.\n')

deepstack_data_path = 'data/official_deepstack_handhistory'
analytics_output = 'data/handhistory_analysis.json'

if os.path.exists(deepstack_data_path):
    print('✓ Official DeepStack data found!')
    
    # Check if analysis already exists
    if os.path.exists(analytics_output):
        print(f'✓ Analysis file already exists: {analytics_output}')
        with open(analytics_output, 'r') as f:
            analysis = json.load(f)
        
        # Display key insights
        if 'insights' in analysis:
            insights = analysis['insights']
            print('\n📊 Key Insights from DeepStack Championship Data:')
            
            if 'street_distribution' in insights:
                print('\n  Street Distribution:')
                for street, pct in insights['street_distribution'].items():
                    print(f'    {street}: {pct*100:.1f}%')
            
            if 'recommended_cfr_iterations' in insights:
                cfr_rec = insights['recommended_cfr_iterations']
                print(f'\n  Recommended CFR Iterations:')
                print(f'    Minimum: {cfr_rec["minimum"]}')
                print(f'    Recommended: {cfr_rec["recommended"]}')
                print(f'    Championship: {cfr_rec["championship"]}')
            
            if 'training_recommendations' in insights:
                print('\n  Training Recommendations:')
                for rec in insights['training_recommendations']:
                    print(f'    • {rec}')
        
        print('\n✓ Ready to use championship-level insights in data generation!')
    else:
        print('\n⚠️  Analysis file not found. Running analysis...')
        print('   This may take a few minutes.\n')
        try:
            !python scripts/analyze_handhistory.py
            print('\n✓ Analysis complete!')
        except Exception as e:
            print(f'⚠️  Error during analysis: {e}')
            print('   Continuing without official data insights.')
else:
    print('⚠️  Official DeepStack data not found at:', deepstack_data_path)
    print('   Continuing with pure CFR generation.\n')
    print('💡 To use championship data:')
    print('   1. Download official DeepStack hand histories')
    print('   2. Place in data/official_deepstack_handhistory/')
    print('   3. Re-run this cell')


In [ ]:
# 4. Generate training data with progress monitoring and official DeepStack data integrationimport timeimport subprocessfrom datetime import datetimeimport osprint('=== DATA GENERATION PHASE ===')print('\nThis notebook can train with two types of data:')print('  1. CFR-generated synthetic data (default)')print('  2. Official DeepStack championship data-informed generation')print('  3. Blended approach (recommended for best results)\n')# Check if official DeepStack data existshas_official_data = os.path.exists('data/official_deepstack_handhistory')if has_official_data:    print('✓ Official DeepStack championship hand history data detected!')    print('  Using analytics-informed bet sizing and street weights\n')    use_analytics = Trueelse:    print('⚠️  Official DeepStack data not found. Using pure CFR generation.\n')    use_analytics = Falseprint('📊 Configuration:')print('  - Samples: 5000 (medium training - balanced quality)')print('  - CFR Iterations: 2000 (high quality)')print('  - Championship bet sizing: Enabled')print(f'  - DeepStack analytics: {"Enabled" if use_analytics else "Disabled"}')print('  - Estimated time: 1.5-2 hours\n')print('💡 TIP: Adjust samples for different training modes:')print('  - Quick test: --profile testing (~10 min)')print('  - Medium (current): --profile development (~2 hours)')print('  - Production: --profile production (~18-24 hours)')print('  - Championship: --profile championship (~4-5 days)\n')start_time = time.time()print(f'⏰ Started at: {datetime.now().strftime("%H:%M:%S")}')print('\nGenerating data... (this will take a while)\n')try:    # Build command with analytics if available    cmd = 'python scripts/generate_data.py --profile development --yes'    if use_analytics:        cmd += ' --use-latest-analytics'        # Run data generation with real-time output    !{cmd}        elapsed = time.time() - start_time    print(f'\n✓ Data generation completed in {elapsed/60:.1f} minutes')        # Verify output    output_dirs = ['src/train_samples_dev', 'src/train_samples']    found_data = False    for output_dir in output_dirs:        if os.path.exists(output_dir):            files = [f for f in os.listdir(output_dir) if f.endswith('.npz')]            if files:                print(f'✓ Generated {len(files)} data files in {output_dir}')                # Store path for training                DATA_PATH = output_dir                found_data = True                break        if not found_data:        print('⚠️  Warning: No data files found in expected directories')        print('  Check the output above for the actual output directory')        DATA_PATH = 'src/train_samples_dev'  # Default        except Exception as e:    print(f'\n❌ Error during data generation: {e}')    print('Check the output above for details.')    DATA_PATH = 'src/train_samples_dev'  # Default fallback

In [ ]:
# 5. Train model with championship configurationimport timefrom datetime import datetimeimport osprint('=== TRAINING PHASE ===')# Determine data path from previous cell or use defaulttry:    data_path = DATA_PATHexcept NameError:    # Find the most recent data directory    possible_paths = ['src/train_samples_dev', 'src/train_samples', 'src/train_samples_analytics']    data_path = None    for path in possible_paths:        if os.path.exists(path) and os.path.isdir(path):            files = [f for f in os.listdir(path) if f.endswith('.npz')]            if files:                data_path = path                break        if not data_path:        print('❌ ERROR: No training data found!')        print('Please run the data generation cell first.')        import sys        sys.exit(1)print(f'\n📂 Using data from: {data_path}')print('\n🎯 Configuration:')print('  - Config: Championship (optimized hyperparameters)')print('  - Epochs: 150 (medium training)')print(f'  - GPU: {"Enabled" if USE_GPU else "Disabled"}')print(f'  - Estimated time: {"1-1.5 hours" if USE_GPU else "4-6 hours"}\n')print('💡 TIP: Adjust epochs for different training modes:')print('  - Quick test: --epochs 50')print('  - Medium (current): --epochs 150')print('  - Production: --epochs 300\n')start_time = time.time()print(f'⏰ Started at: {datetime.now().strftime("%H:%M:%S")}')print('\nTraining... (watch for progress updates)\n')try:    # Build command based on GPU availability    gpu_flag = '--use-gpu' if USE_GPU else ''    cmd = f'python scripts/train_deepstack.py --data {data_path} --config scripts/config/championship.json {gpu_flag} --epochs 150'        # Run training with real-time output    !{cmd}        elapsed = time.time() - start_time    print(f'\n✓ Training completed in {elapsed/60:.1f} minutes')        # Check for model file    model_paths = ['models/versions/best_model.pt', 'models/best_model.pt']    model_found = False    for model_path in model_paths:        if os.path.exists(model_path):            size_mb = os.path.getsize(model_path) / (1024*1024)            print(f'✓ Model saved: {model_path} ({size_mb:.1f} MB)')            model_found = True            break        if not model_found:        print('⚠️  Warning: Model file not found at expected location')        print('  Training may have encountered an issue.')        except Exception as e:    print(f'\n❌ Error during training: {e}')    print('Check the output above for details.')

In [ ]:
# 6. Validate model performance
import os

print('=== VALIDATION PHASE ===')
print('\n📊 Checking model quality metrics...\n')

try:
    # Check if model exists
    model_path = 'models/versions/best_model.pt'
    if not os.path.exists(model_path):
        print(f'❌ Model not found at {model_path}')
        print('Please ensure training completed successfully.')
    else:
        # Run validation
        !python scripts/validate_deepstack_model.py --model models/versions/best_model.pt
        
        print('\n=== INTERPRETATION GUIDE ===')
        print('✅ GOOD: Correlation > 0.85, Relative Error < 5%')
        print('⚠️  NEEDS IMPROVEMENT: Correlation 0.5-0.85, Relative Error 5-20%')
        print('❌ POOR: Correlation < 0.5, Relative Error > 20%')
        print('\n💡 If results are poor, try:')
        print('   1. Generate more data (--samples 10000)')
        print('   2. Train for more epochs (--epochs 300)')
        print('   3. Use adaptive bucket weighting')
        
except Exception as e:
    print(f'❌ Error during validation: {e}')

## 📈 Results Analysis and Visualization

The cells below provide detailed analysis and visualization of your trained model:

### What to Expect:
- **Loss Curves**: Training vs validation loss over epochs
- **Correlation Plots**: How well predictions match targets
- **Per-Street Analysis**: Performance breakdown by game stage
- **Per-Bucket Correlations**: Which hand types the model understands best

### Next Steps:
1. Review validation metrics above
2. Examine visualizations below
3. If performance is good (correlation > 0.85), download the model
4. If performance needs improvement, see troubleshooting tips

### Troubleshooting Tips:
- **Low correlation?** → Regenerate data with more samples or higher CFR iterations
- **High variance?** → Train for more epochs or increase batch size
- **Poor on specific streets?** → Check street coverage in data generation
- **Some buckets poor?** → Use adaptive bucket weighting (see advanced cells)

In [ ]:
# 7. Generate and display analysis reports
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display

print('=== GENERATING ANALYSIS REPORTS ===')
print('\nCreating visualizations...\n')

try:
    # Generate analysis report
    !python scripts/run_analysis_report.py
    
    print('\n=== VISUALIZATION PLOTS ===\n')
    
    # List of expected plots
    plots = [
        ('Loss Curves', 'models/reports/loss_curves.png'),
        ('Correlation Analysis', 'models/reports/correlation_curves.png'),
        ('Per-Street Performance', 'models/reports/per_street_correlation.png'),
        ('Per-Bucket Histogram', 'models/reports/per_bucket_corrs_hist.png')
    ]
    
    # Display each plot with error handling
    displayed_count = 0
    for title, plot_path in plots:
        try:
            if os.path.exists(plot_path):
                print(f'📊 {title}')
                display(Image(plot_path))
                displayed_count += 1
            else:
                print(f'⚠️  {title}: Not found at {plot_path}')
        except Exception as e:
            print(f'❌ Error displaying {title}: {e}')
    
    print(f'\n✓ Displayed {displayed_count}/{len(plots)} plots')
    
    if displayed_count == 0:
        print('\n💡 No plots were generated. This might mean:')
        print('   - Analysis script needs the model to be trained first')
        print('   - There was an error in report generation')
        print('   - Check the output above for error messages')
        
except Exception as e:
    print(f'\n❌ Error generating analysis: {e}')
    print('You can still download the model and use it, but visualizations are unavailable.')

---
## 🔧 Advanced Features (Optional)

The cells below provide advanced functionality for power users:
- Interactive metric visualization with detailed statistics
- Model and results download for local use

**Note:** These are optional and only needed for deeper analysis or custom workflows.

In [ ]:
# 8. Advanced: Interactive metric visualization
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json

print('=== INTERACTIVE METRICS VISUALIZATION ===\n')

# Configure matplotlib for better display
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# 1. Training/Validation Loss Curves
metrics_path = 'models/reports/training_metrics.csv'
if os.path.exists(metrics_path):
    print('📊 Training Metrics Found')
    try:
        df = pd.read_csv(metrics_path)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Loss curves
        train_data = df[df['phase'] == 'train']
        val_data = df[df['phase'] == 'valid']
        
        ax1.plot(train_data['epoch'], train_data['loss'], 
                label='Training Loss', linewidth=2, alpha=0.8)
        ax1.plot(val_data['epoch'], val_data['loss'], 
                label='Validation Loss', linewidth=2, alpha=0.8)
        ax1.set_xlabel('Epoch', fontsize=12)
        ax1.set_ylabel('Loss', fontsize=12)
        ax1.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')
        ax1.legend(fontsize=11)
        ax1.grid(True, alpha=0.3)
        
        # Correlation over time (if available)
        if 'correlation' in df.columns:
            ax2.plot(val_data['epoch'], val_data['correlation'], 
                    linewidth=2, color='green', alpha=0.8)
            ax2.axhline(y=0.85, color='r', linestyle='--', 
                       label='Target (0.85)', alpha=0.7)
            ax2.set_xlabel('Epoch', fontsize=12)
            ax2.set_ylabel('Correlation', fontsize=12)
            ax2.set_title('Validation Correlation Over Time', 
                         fontsize=14, fontweight='bold')
            ax2.legend(fontsize=11)
            ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print summary statistics
        print('\n📈 Summary Statistics:')
        print(f'  Final Training Loss: {train_data["loss"].iloc[-1]:.4f}')
        print(f'  Final Validation Loss: {val_data["loss"].iloc[-1]:.4f}')
        if 'correlation' in df.columns:
            print(f'  Final Correlation: {val_data["correlation"].iloc[-1]:.4f}')
        
    except Exception as e:
        print(f'❌ Error plotting metrics: {e}')
else:
    print(f'⚠️  Training metrics not found at {metrics_path}')

print('\n' + '='*50 + '\n')

# 2. Per-Bucket Correlation Distribution
corr_path = 'models/reports/per_bucket_corrs.json'
if os.path.exists(corr_path):
    print('📊 Per-Bucket Correlations Found')
    try:
        with open(corr_path) as f:
            data = json.load(f)
        
        bucket_corrs = np.array(data.get('bucket_corrs', []))
        
        if len(bucket_corrs) > 0:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
            
            # Histogram
            ax1.hist(bucket_corrs, bins=30, alpha=0.7, color='skyblue', 
                    edgecolor='black')
            ax1.axvline(x=bucket_corrs.mean(), color='r', linestyle='--',
                       label=f'Mean: {bucket_corrs.mean():.3f}', linewidth=2)
            ax1.axvline(x=0.3, color='g', linestyle='--',
                       label='Target: 0.3', linewidth=2, alpha=0.7)
            ax1.set_xlabel('Correlation', fontsize=12)
            ax1.set_ylabel('Count', fontsize=12)
            ax1.set_title('Distribution of Per-Bucket Correlations', 
                         fontsize=14, fontweight='bold')
            ax1.legend(fontsize=11)
            ax1.grid(True, alpha=0.3, axis='y')
            
            # Box plot
            ax2.boxplot(bucket_corrs, vert=True)
            ax2.axhline(y=0.3, color='g', linestyle='--', 
                       label='Target: 0.3', alpha=0.7)
            ax2.set_ylabel('Correlation', fontsize=12)
            ax2.set_title('Per-Bucket Correlation Statistics', 
                         fontsize=14, fontweight='bold')
            ax2.legend(fontsize=11)
            ax2.grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
            
            # Statistics
            print('\n📈 Per-Bucket Statistics:')
            print(f'  Mean Correlation: {bucket_corrs.mean():.4f}')
            print(f'  Median Correlation: {np.median(bucket_corrs):.4f}')
            print(f'  Std Deviation: {bucket_corrs.std():.4f}')
            print(f'  Min: {bucket_corrs.min():.4f}, Max: {bucket_corrs.max():.4f}')
            print(f'  Buckets with Corr > 0.3: {(bucket_corrs > 0.3).sum()}/{len(bucket_corrs)}')
        
    except Exception as e:
        print(f'❌ Error: {e}')
else:
    print(f'⚠️  Per-bucket correlations not found')

print('\n✓ Visualization complete')

In [ ]:
# 9. Download trained model and results
import shutil
import os
from datetime import datetime

try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print('⚠️  Not running in Google Colab - download functionality disabled')

if IN_COLAB:
    print('=== DOWNLOAD ARTIFACTS ===\n')
    print('Preparing files for download...\n')
    
    # Create timestamp for unique filename
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    archive_name = f'pokerbot_artifacts_{timestamp}'
    
    try:
        # Check what we have
        items_to_include = []
        
        if os.path.exists('models/versions/best_model.pt'):
            items_to_include.append('Trained model (best_model.pt)')
        
        if os.path.exists('models/reports'):
            report_files = os.listdir('models/reports')
            items_to_include.append(f'Analysis reports ({len(report_files)} files)')
        
        if os.path.exists('scripts/config/championship.json'):
            items_to_include.append('Championship config')
        
        print('📦 Package contents:')
        for item in items_to_include:
            print(f'  ✓ {item}')
        
        print(f'\nCreating archive: {archive_name}.zip...')
        
        # Create temporary directory for packaging
        os.makedirs('download_package', exist_ok=True)
        
        # Copy files to package directory
        if os.path.exists('models'):
            shutil.copytree('models', 'download_package/models', dirs_exist_ok=True)
        
        if os.path.exists('scripts/config'):
            shutil.copytree('scripts/config', 'download_package/config', dirs_exist_ok=True)
        
        # Create README for the package
        with open('download_package/README.txt', 'w') as f:
            f.write('PokerBot Training Artifacts\n')
            f.write('='*50 + '\n\n')
            f.write(f'Generated: {datetime.now()}\n\n')
            f.write('Contents:\n')
            for item in items_to_include:
                f.write(f'  - {item}\n')
            f.write('\nUsage:\n')
            f.write('  1. Extract this archive\n')
            f.write('  2. Model file: models/versions/best_model.pt\n')
            f.write('  3. Reports: models/reports/\n')
            f.write('  4. Config: config/championship.json\n')
        
        # Create ZIP archive
        shutil.make_archive(archive_name, 'zip', 'download_package')
        
        # Get file size
        file_size = os.path.getsize(f'{archive_name}.zip') / (1024 * 1024)
        print(f'✓ Archive created: {archive_name}.zip ({file_size:.1f} MB)')
        
        # Trigger download
        print('\n⬇️  Downloading...')
        files.download(f'{archive_name}.zip')
        print('✓ Download started!')
        
        # Cleanup
        shutil.rmtree('download_package')
        
        print('\n✅ Download complete!')
        print('\n💡 Next steps:')
        print('   1. Extract the ZIP file on your computer')
        print('   2. Use the model in your poker application')
        print('   3. Review the analysis reports for insights')
        
    except Exception as e:
        print(f'\n❌ Error creating download package: {e}')
        print('\n💡 Alternative: Manually download files from the Files panel')
else:
    print('\n💡 To download files in local Jupyter:')
    print('   - Navigate to the Files panel')
    print('   - Right-click on models/ folder → Download')

---
## 📊 Understanding Your Results

### Validation Metrics Explained

**Correlation (Most Important)**
- Measures how well predictions match actual values
- **>0.85**: Excellent - Championship quality
- **0.70-0.85**: Good - Production ready
- **0.50-0.70**: Fair - Needs improvement
- **<0.50**: Poor - Increase samples or CFR iterations

**Relative Error**
- Average percentage difference from true values
- **<5%**: Excellent accuracy
- **5-20%**: Acceptable for most applications
- **>20%**: Needs more training

**Loss Curves**
- Should decrease steadily during training
- Validation loss should track training loss
- Large gap = overfitting (reduce epochs or add regularization)
- Flat lines = converged (good!)

### Per-Street Performance

Different streets have different difficulty levels:
- **Preflop**: Usually easiest (limited game tree)
- **Flop**: More complex (texture matters)
- **Turn**: Intermediate complexity
- **River**: Most complex (full information available)

All streets should have correlation >0.70 for production use.

### What Data Source Was Used?

Check the data generation output:
- **"DeepStack analytics: Enabled"** = Using championship insights
- **"Championship bet sizing: Enabled"** = Using pot-relative abstractions
- **"DeepStack analytics: Disabled"** = Pure CFR generation

Championship data insights typically improve correlation by 5-10%.

### Troubleshooting Poor Results

**Low Overall Correlation (<0.65)**
1. Regenerate with more samples: `--profile production`
2. Increase CFR iterations: `--cfr-iters 2500`
3. Enable analytics if available: `--use-latest-analytics`
4. Train for more epochs: `--epochs 300`

**High Validation Loss**
1. Check for overfitting (validation >> training loss)
2. Reduce batch size or add dropout
3. Ensure enough training samples

**Specific Street Performing Poorly**
1. Generate more samples focusing on that street
2. Check street weight distribution in analytics
3. May need more CFR iterations for complex streets

**GPU Out of Memory**
1. Reduce batch size: Edit `config/championship.json`
2. Use CPU training (slower but works)
3. Reduce model size (advanced)

### Next Steps After Training

**Download Your Model**
- Run cell 9 to package and download
- Includes model, reports, and configuration

**Evaluate Performance**
- Review all visualization plots
- Check per-bucket correlations
- Validate against held-out test data

**Deploy Your Model**
- Model file: `models/versions/best_model.pt`
- Load with PyTorch for inference
- Use with DeepStack continual resolving

**Iterate and Improve**
- If results are good: Try production profile
- If results are poor: Adjust parameters and retrain
- Track improvements over time

### Performance Benchmarks

Based on our testing with championship data:

| Configuration | Validation Correlation | Training Time | Model Quality |
|--------------|----------------------|---------------|---------------|
| Testing (1K) | 0.45-0.55 | 10 min | Demo only |
| Development (10K) | 0.70-0.75 | 2 hours | Good |
| Production (100K) | 0.80-0.85 | 24 hours | Excellent |
| Championship (500K) | 0.85-0.90 | 5 days | World-class |

*With championship data analytics enabled*

---

**Questions or Issues?**
- Check the GitHub repository: https://github.com/elliotttmiller/pokerbot
- Review documentation in `/docs/`
- Open an issue for bugs or feature requests

---
## 📚 Additional Resources and Quick Reference

### 🎯 Command Quick Reference

**Data Generation Profiles**
```bash
# Quick test (~10 min)
python scripts/generate_data.py --profile testing --yes

# Development (~2 hours)
python scripts/generate_data.py --profile development --yes

# Production with analytics (~24 hours)
python scripts/generate_data.py --profile production --use-latest-analytics --yes

# Championship (~5 days)
python scripts/generate_data.py --profile championship --adaptive-cfr --yes
```

**Training Commands**
```bash
# Standard training
python scripts/train_deepstack.py --data src/train_samples_dev --use-gpu --epochs 150

# Long training for better results
python scripts/train_deepstack.py --data src/train_samples_production --use-gpu --epochs 300
```

### 🔧 Configuration Files

- `scripts/config/championship.json`: Training hyperparameters
- `config/data_generation/parameters/`: Analytics-derived parameters
- `data/handhistory_analysis.json`: Championship data insights

### 📖 Documentation

Located in `/docs/` directory:
- `DEEPSTACK_OFFICIAL_ANALYSIS.md`: Analysis of official DeepStack implementation
- `QUICK_REFERENCE_DEEPSTACK.md`: Quick start guide
- `DEEPSTACK_IMPROVEMENTS_SUMMARY.md`: Recent improvements

### 🐛 Common Issues

**1. "No training data found"**
- Ensure data generation cell completed successfully
- Check `src/train_samples_dev/` or `src/train_samples/` directories

**2. "CUDA out of memory"**
- Reduce batch size in `config/championship.json`
- Use CPU training (remove `--use-gpu` flag)

**3. "Low correlation (<0.65)"**
- Increase samples: `--profile production`
- Increase CFR iterations: `--cfr-iters 2500`
- Enable analytics: `--use-latest-analytics`

**4. "Training not progressing"**
- Check GPU is enabled (cell 3)
- Verify data was generated (cell 4)
- Look for error messages in output

### 💡 Pro Tips

1. **Start Small**: Always test with `--profile testing` first
2. **Use Analytics**: Championship data improves results by 5-10%
3. **Monitor GPU**: Watch memory usage during training
4. **Save Often**: Download your models regularly
5. **Track Metrics**: Keep a log of configurations and results

### 🔗 Useful Links

- **DeepStack Paper**: https://www.science.org/doi/10.1126/science.aam6960
- **GitHub Repo**: https://github.com/elliotttmiller/pokerbot
- **CFR Explained**: https://poker.cs.ualberta.ca/publications/NIPS07-cfr.pdf

---

**Happy Training! 🎰🤖**

*This notebook uses state-of-the-art poker AI techniques combined with championship-level data analysis.*